25 espacios conectados con lineas verticales, horizontales y diagonales

4 fichas negras y 4 piezas rojas

Si un jugador forma una linea de 4 fichas Gana la partida (Horizontal, Diagonal y Vertical)

Si un jugador forma un cuadrado Gana la partida

Una vez asignadas las 4 piezas el jugador puede seleccionar una y moverla a una casilla sin asignacion

link

los jugadores sortean quien comienza, por defecto comienza el negro y se alternan entre sí

####################################primera fase####################################################################

En la primera parte los jugadores ponen sus marcadores, y cada marcador ocupa un campo, para el final de esta fase hay 8 fichas en el tablero, 4 negras y 4 rojas

Es posible ganar en la primera fase, si nadie gana, se pasa a la segunda fase

####################################segunda fase####################################################################

No se puede saltar turnos

en cada turno se puede mover las fichas en un campo adyacente vacio, ya sea vertical, horizontal o diagonal

####################################objective#######################################################################

organizar las fichas hasta formar un Teeko, existen 44 Teekos distintos, las mismas se dividen en verticales, horizontales , diagonales y cuadradas

To Do List:

- Mejorar la Heuristica tomando en cuanta las piezas del jugador ya sea min o max que esten en el tablero. Asignar un mejor valor cuando un jugador gana
- Realizar un debug al algoritmo min max with depth para ver los fallos
- Agregar cambio de jugadores al min max
- Mejorar el codigo en general
- Hacer el algoritmo min max with depth sin el alpha beta pruning
- Comparar algoritmos y plottear resultados

In [723]:
import numpy as np
import random
from numpy import inf

move_dict = {
  "A": 0,
  "B": 1,
  "C": 2,
  "D": 3,
  "E": 4,
}

class Teeko:
    def __init__(self, player1, player2):
        self.player1 = (player1, 1)  # Red Player
        self.player2 = (player2, 2)  # Black Player
        self.board = np.zeros((5,5))
        self.turn = 0
        self.rand_player()
    
    def rand_player(self):
        n_player = random.randint(1,2) #The red marks have the value 1 and the black marks have the value 2
        if n_player == 1:
            self.current_player = self.player1
        else: self.current_player = self.player2
    
    def switch_player(self):
        if self.current_player == self.player1:
            self.current_player = self.player2
        else: self.current_player = self.player1
        self.turn += 1
        
    def move_translator(self, selected):       #A5, B6 etc.
        yPos = selected[0]
        xPos = int(selected[1]) - 1
        return (move_dict[yPos], xPos)

    def make_a_move(self, selected, move = None):       # selected recibe posiciones reales tupla, move recibe la dirección  # move
        y, x = selected
        valid_move = True
        if move is not None:
            for pos in move:
                if pos == "u" and y > 0:
                    y -= 1
                elif pos == "d" and y < 4:
                    y += 1
                elif pos == "r" and x < 4:
                    x += 1 
                elif pos == "l" and x > 0: 
                    x -= 1
                else: 
                    valid_move = False
        if valid_move and self.board[y][x] == 0:
            self.board[selected[0]][selected[1]] = 0
            limit = x >=0 and x < 5 and y >= 0 and y < 5
            if limit:
                selected = (y, x)
                self.board[selected[0]][selected[1]] = self.current_player[1]
            else: 
                print("Limites superados o posición ocupada")
                valid_move = False
        else: 
            print("Movimiento no válido, vuelva a intentarlo")   
            valid_move = False
            
        return valid_move, selected
        
    def check_horizontal(self, y):
        winner = False
        count = 0
        for row in self.board[y]:
            if row == self.current_player[1]:
                count += 1
            else: count = 0
            if count == 4:
                winner = True
        return winner

    def check_vertical(self, x):
        winner = False
        count = 0
        for col in range(5):
            if self.board[col][x] == self.current_player[1]:
                count += 1
            else: count = 0
            if count == 4:
                winner = True
        return winner
    
    def check_diagonal(self, matrix):
        winner = False
        count = 0
        for k in range(3):
            arr = np.diag(matrix, k - 1)
            for x in arr:
                if x == self.current_player[1]:
                    count += 1
                else: count = 0
                if count == 4:
                    winner = True
            if winner:
                break
        return winner
    
    def check_square(self, x, y):
        winner = False
        count = 0
        for i in range (2):
            if y + 1 < 5 and x + i < 5:
                if self.board[y][x + i] == self.current_player[1]:
                    count += 1
                if self.board[y + 1][x + i] == self.current_player[1]:
                    count += 1
        if count != 4:
            count = 0
            for i in range (2):
                if y - 1 >= 0 and x + i < 5:
                    if self.board[y][x + i] == self.current_player[1]:
                        count += 1
                    if self.board[y - 1][x + i] == self.current_player[1]:
                        count += 1
        if count != 4:
            count = 0
            for i in range (2):
                if y + 1 < 5 and x - i >= 0:
                    if self.board[y][x - i] == self.current_player[1]:
                        count += 1
                    if self.board[y + 1][x - i] == self.current_player[1]:
                        count += 1
        if count != 4:
            count = 0
            for i in range (2):
                if y - 1 >= 0 and x - i >= 0:
                    if self.board[y][x - i] == self.current_player[1]:
                        count += 1
                    if self.board[y - 1][x - i] == self.current_player[1]:
                        count += 1
        if count == 4:
            winner = True
        return winner

    def check_winner(self, last_position): #0,1 2,0
        y, x = last_position
        count = 0
        winner = False
        winner = self.check_horizontal(y)
        if not winner:
            winner = self.check_vertical(x)
        if not winner:
            winner = self.check_diagonal(self.board)
        if not winner:
            winner = self.check_diagonal(np.fliplr(self.board))
        if not winner:
            winner = self.check_square(x, y)
        return winner  
    
    def get_position(self):
        arr_pos = []
        for y in range(5):
            for x in range(5):
                if self.board[y][x] == self.current_player[1]:
                    arr_pos.append((y,x))
        return arr_pos
    
    def get_actions(self, selected):
        y, x = selected
        actions = ['u', 'd', 'l', 'r', 'ul', 'ur', 'dl', 'dr']
        valid_move = True
        valid_actions = []
        for action in actions:
            valid_move = True
            y, x = selected
            for pos in action:
                if pos == "u" and y > 0:
                    y -= 1
                elif pos == "d" and y < 4:
                    y += 1
                elif pos == "r" and x < 4:
                    x += 1 
                elif pos == "l" and x > 0: 
                    x -= 1
                else: 
                    valid_move = False
            if valid_move and self.board[y][x] == 0:
                valid_actions.append(action)
        return valid_actions
    
    def heuristic(self, position):
        strength = 0
        mat = [[4,6,5,6,4],[6,10,10,10,6],[5,10,12,10,5],[6,10,10,10,6],[4,6,5,6,4]]
        
        for y in range(5):
            for x in range(5):
                if self.board[y][x] == self.player1[1]:
                    strength += mat[y][x]
                if self.board[y][x] == self.player2[1]:
                    strength -= mat[y][x]
        if self.current_player[0] == "MAX":
            strength -= self.turn * 2
        if self.current_player[0] == "MIN":
            strength += self.turn * 2
        if self.check_winner(position):
            if self.current_player[0] == "MAX":
                strength += 100
            if self.current_player[0] == "MIN":
                strength -= 100
        return strength

In [724]:
#Minimax Algorithm

def minmax_decision(game):  #player MIN MAX, game es de la clase Teeko
    values = []
    i = 0
    mark_position = game.get_position() # el get devuelve un array de tuplas de las posiciones de todas las fichas del current_player 
    actions = []
    moves = []
    if game.current_player[0] == "MAX":
        for mark in mark_position:
            actions = game.get_actions(mark)
            for action in actions:
                b, selected = game.make_a_move(mark, action)
                value = min_value(game,selected)
                print(game.board)
                game.board[mark[0]][mark[1]] = 1
                game.board[selected[0]][selected[1]] = 0
                values.append(value)
                moves.append(action)
        i = np.argmax(values)
    else:
        for mark in mark_position:
            actions = game.get_actions(mark)
            for action in actions:
                b, selected = game.make_a_move(mark, action)
                value = max_value(game,selected)
                game.board[mark[0]][mark[1]] = 2
                game.board[selected[0]][selected[1]] = 0
                values.append(value)
        i = np.argmin(values)
    return moves[i]


def min_value(game, last_position):
    mark_position = game.get_position() # el get devuelve un array de tuplas de las posiciones de todas las fichas del current_player 
    actions = []
    value = inf
    if game.check_winner(last_position):
        value = game.heuristic() #nos falta la heurística
    else:
        for mark in mark_position:
            actions = game.get_actions(mark)
            for action in actions:
                b, selected = game.make_a_move(mark, action) 
                value = min(value, max_value(game, selected))
                game.board[mark[0]][mark[1]] = 2
                game.board[selected[0]][selected[1]] = 0
    return value

def max_value(game, last_position):
    mark_position = game.get_position() # el get devuelve un array de tuplas de las posiciones de todas las fichas del current_player
    actions = []
    value = -inf
    if game.check_winner(last_position):
        value = game.heuristic() #nos falta la heurística
    else:
        for mark in mark_position:
            actions = game.get_actions(mark)
            for action in actions: 
                b, selected = game.make_a_move(mark, action)
                value = max(value, min_value(game, selected))
                game.board[mark[0]][mark[1]] = 1
                game.board[selected[0]][selected[1]] = 0
    return value

def min(value1, value2): #return the minimum value
    if value1 < value2:
        return value1
    else:
        return value2

def max(value1, value2): #return the max value
    if value1 > value2:
        return value1
    else:
        return value2

In [725]:
#Alpha Beta Pruning Algorithm

def alpha_beta_search(game):  #player MIN MAX, game es de la clase Teeko
    values = []
    i = 0
    mark_position = game.get_position() # el get devuelve un array de tuplas de las posiciones de todas las fichas del current_player 
    actions = []
    moves = []
    if game.current_player[0] == "MAX":
        for mark in mark_position:
            actions = game.get_actions(mark)
            for action in actions:
                valid_move, selected = game.make_a_move(mark, action)
                value = min_value_pruning(game, selected,-inf, inf)
                game.board[mark[0]][mark[1]] = 1
                game.board[selected[0]][selected[1]] = 0
                values.append(value) 
                moves.append(action)
        i = np.argmax(values)
    else:
        for mark in mark_position:
            actions = game.get_actions(mark)
            for action in actions:
                b, selected = game.make_a_move(mark, action)
                value = max_value_pruning(game, selected,-inf, inf)
                game.board[mark[0]][mark[1]] = 2
                game.board[selected[0]][selected[1]] = 0
                values.append(value) 
                moves.append(action)
        i = np.argmin(values)
    return moves[i]

def min_value_pruning(self, game, last_position, alpha, beta):
    mark_position = game.get_position() # el get devuelve un array de tuplas de las posiciones de todas las fichas del current_player 
    actions = []
    v = inf
    if game.check_winner(last_position):
        return game.heuristic(last_position)
    else: 
        for mark in mark_position:
            actions = game.get_actions(mark)
            for action in actions:
                b, selected = game.make_a_move(mark, action)
                v = min(v, max_value_pruning(game, selected,alpha, beta))
                game.board[mark[0]][mark[1]] = 2
                game.board[selected[0]][selected[1]] = 0
                if v <= alpha:
                    return v
                beta = min(beta,v)
    return v

def max_value_pruning(self, game, last_position, alpha, beta):
    mark_position = game.get_position() # el get devuelve un array de tuplas de las posiciones de todas las fichas del current_player 
    actions = []
    v = -inf
    if game.check_winner(last_position):
        return game.heuristic(last_position)
    else: 
        for mark in mark_position:
            actions = game.get_actions(mark)
            for action in actions:
                b, selected = game.make_a_move(mark, action)
                v = max(v, min_value_pruning(game, selected,alpha, beta))
                game.board[mark[0]][mark[1]] = 1
                game.board[selected[0]][selected[1]] = 0 
                if v >= beta:
                    return v
                alpha = max(alpha,v)
    return v


In [726]:
#Minimax with depth Algorithm

from copy import deepcopy


max_depth = 1

def min_max_with_depth(game):  #player MIN MAX, game es de la clase Teeko
    values = []
    i = 0
    mark_position = game.get_position() # el get devuelve un array de tuplas de las posiciones de todas las fichas del current_player 
    actions = []
    moves = []
    pieces = []
    if game.current_player[0] == "MAX":
        for mark in mark_position:
            actions = game.get_actions(mark)
            for action in actions:
                copyGame = deepcopy(game)
                b, selected = copyGame.make_a_move(mark, action)
                value = min_value_with_depth(copyGame, selected,-inf, inf, 0)
                #game.board[mark[0]][mark[1]] = 1
                #game.board[selected[0]][selected[1]] = 0
                values.append(value) 
                moves.append(action)
                pieces.append(mark)
        i = np.argmax(values)
    else:
        for mark in mark_position:
            actions = game.get_actions(mark)
            for action in actions:
                copyGame = deepcopy(game)
                b, selected = copyGame.make_a_move(mark, action)
                value = max_value_with_depth(copyGame, selected,-inf, inf, 0)
                #game.board[mark[0]][mark[1]] = 2
                #game.board[selected[0]][selected[1]] = 0
                values.append(value) 
                moves.append(action)
                pieces.append(mark)
        i = np.argmin(values)
    return moves[i], pieces[i]

def min_value_with_depth(game, last_position, alpha, beta, depth):    
    actions = []
    v = inf
    if cut_off(game, last_position, depth):
        v = game.heuristic(last_position)
        game.switch_player()
        return v
    else: 
        game.switch_player()
        mark_position = game.get_position() # el get devuelve un array de tuplas de las posiciones de todas las fichas del current_player 
        for mark in mark_position:
            actions = game.get_actions(mark)
            for action in actions:
                copyGame = deepcopy(game)
                b, selected = copyGame.make_a_move(mark, action)
                v = min(v, max_value_with_depth(copyGame, selected,alpha, beta, depth + 1))
                #game.board[mark[0]][mark[1]] = 2
                #game.board[selected[0]][selected[1]] = 0
                if v <= alpha:
                    return v
                beta = min(beta,v)
    game.switch_player()
    return v

def max_value_with_depth(game, last_position, alpha, beta, depth):
    actions = []
    v = -inf
    if cut_off(game, last_position, depth) :
        v = game.heuristic(last_position)
        game.switch_player()
        return v
    else: 
        game.switch_player()
        mark_position = game.get_position() # el get devuelve un array de tuplas de las posiciones de todas las fichas del current_player 
        for mark in mark_position:
            actions = game.get_actions(mark)
            for action in actions:
                copyGame = deepcopy(game)
                b, selected = copyGame.make_a_move(mark, action)
                v = max(v, min_value_with_depth(copyGame, selected,alpha, beta, depth + 1))
                #game.board[mark[0]][mark[1]] = 1
                #game.board[selected[0]][selected[1]] = 0 
                if v >= beta:
                    return v
                alpha = max(alpha,v)
    game.switch_player()
    return v

def cut_off(game, last_position, depth):
    return depth == max_depth or game.check_winner(last_position)

In [727]:
#Juego
from IPython.display import clear_output
import matplotlib.pyplot as plot

#El Codigo aca esta horrible, but it works Xd

def gameloop():
    winner_found = False
    game1 = Teeko("MAX", "MIN")
    plot.matshow(game1.board)
    plot.show()
    #print(game1.board)
    turns = 1
    while winner_found != True:
        print("=====================")
        print(f"El jugador actual es: {game1.current_player}")
        if turns > 8:
            if turns % 2 == 0:
                move, selected_piece = min_max_with_depth(deepcopy(game1))
                clear_output(wait=True)
                print("=====================")
                print(f"El jugador actual es: {game1.current_player}")
                print(f"Turno: {turns}")
                print(f"Ficha seleccionada a {selected_piece}")
                print(f"Movimiento a {move}")
                valid_move, selected = game1.make_a_move(selected_piece, move)
                winner_found = game1.check_winner(selected)
            else:
                print(f"Jugador {game1.current_player[0]} seleccione una ficha e ingrese un movimiento")
                selected_piece = input()
                move = input()
                clear_output(wait=True)
                print(f"Ficha seleccionada a {selected_piece}")
                print(f"Movimiento a {move}")
                selected_translated = game1.move_translator(selected_piece)
                valid_move, selected = game1.make_a_move(selected_translated, move)
                winner_found = game1.check_winner(selected)
        else:
            print(f"Jugador {game1.current_player[0]} ingrese un movimiento")
            move = input()
            clear_output(wait=True)
            print(f"Movimiento a {move}")
            move_translated = game1.move_translator(move)
            valid_move, selected = game1.make_a_move(move_translated)
            winner_found = game1.check_winner(move_translated)
        plot.matshow(game1.board)
        plot.show()
        if not winner_found and valid_move:                             #Si el jugador realiza un movimiento invalido repite su turno
                game1.switch_player()
                turns += 1
    print(f"El jugador ganador es: {game1.current_player}")
    return 0


In [728]:
#gameloop()

In [729]:
game = Teeko("Fernando", "Bruno")
print(game.current_player)
print(game.board)

('Bruno', 2)
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


In [730]:
game.switch_player()
print(game.current_player)
print(game.move_translator("C5"))

('Fernando', 1)
(2, 4)


In [731]:
game.make_a_move((0,0))
print(game.board)

[[1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


In [732]:
game.make_a_move((0,1))
game.make_a_move((1,0))
game.make_a_move((1,1))
print(game.check_square(0,0))
print(game.board)

True
[[1. 1. 0. 0. 0.]
 [1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


In [733]:
gameTest = Teeko("MAX", "MIN")
gameTest.make_a_move((0,0))
gameTest.switch_player()

gameTest.make_a_move((2,2))
gameTest.switch_player()

gameTest.make_a_move((0,4))
gameTest.switch_player()

gameTest.make_a_move((2,3))
gameTest.switch_player()

gameTest.make_a_move((4,0))
gameTest.switch_player()

gameTest.make_a_move((1,3))
gameTest.switch_player()

gameTest.make_a_move((4,4))
gameTest.switch_player()

gameTest.make_a_move((0,2))
gameTest.switch_player()

gameTest.make_a_move((4,3))
gameTest.switch_player()

print(gameTest.board)
gameTest.get_actions((2,1))
print(min_max_with_depth(gameTest))
#print(gameTest.current_player)
#gameTest.get_position()

[[2. 0. 1. 0. 2.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 1. 0.]
 [0. 0. 0. 0. 0.]
 [2. 0. 0. 2. 2.]]
('d', (0, 2))


In [734]:
gameTest2 = Teeko("MAX", "MIN")

print("Primer Jugador: ", gameTest2.current_player)
gameTest2.make_a_move((4,2))
gameTest2.switch_player()

gameTest2.make_a_move((1,0))
gameTest2.switch_player()

gameTest2.make_a_move((3,3))
gameTest2.switch_player()

gameTest2.make_a_move((1,4))
gameTest2.switch_player()

gameTest2.make_a_move((2,3))
gameTest2.switch_player()

gameTest2.make_a_move((4,0))
gameTest2.switch_player()

gameTest2.make_a_move((2,1))
gameTest2.switch_player()

gameTest2.make_a_move((1,2))
gameTest2.switch_player()

gameTest2.make_a_move((4,2),"u")
gameTest2.switch_player()


print(gameTest2.board)
#gameTest.get_actions((2,1))
print(min_max_with_depth(deepcopy(gameTest2)))
print(gameTest2.board)

Primer Jugador:  ('MAX', 1)
[[0. 0. 0. 0. 0.]
 [2. 0. 2. 0. 2.]
 [0. 1. 0. 1. 0.]
 [0. 0. 1. 1. 0.]
 [2. 0. 0. 0. 0.]]
('d', (1, 2))
[[0. 0. 0. 0. 0.]
 [2. 0. 2. 0. 2.]
 [0. 1. 0. 1. 0.]
 [0. 0. 1. 1. 0.]
 [2. 0. 0. 0. 0.]]
